<a href="https://colab.research.google.com/github/cu-applied-math/appm-4600-numerics/blob/main/Demos/Ch1_ExponentialSummation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [Ch1_SymbolicTaylorSeries](Ch1_SymbolicTaylorSeries.ipynb) we saw how to use the Taylor remainder theorem to estimate error in an expansion, and this is useful if you want to calculate a function like sine or cosine using only additions/subtractions and multiplications/division.

Let's try that with the exponential function.  This is a classic example in numerical analysis

In [13]:
# Load the package
import numpy as np
import sympy as sym
from sympy import init_printing
init_printing()  # This will make output look very nice!

import math  # has things like math.pi (which is the numerical value of pi)

In [2]:
# First, all symbolic variables must be declared. This is in contrast to Mathematica
x = sym.Symbol('x')

# Recall the Taylor series for exp(x)... very simple!
sym.series( sym.exp(-x), x )

         2    3    4    5         
        x    x    x    x      ⎛ 6⎞
1 - x + ── - ── + ── - ─── + O⎝x ⎠
        2    6    24   120        

Let's try to find $e^{-30}$ with about 14 digits of precision.  Recally that Taylor's remainder theorem says that the error in the Taylor series (with $n-1$ terms) is the $n$th term but evaluate the derivative at some $\xi \in [x_0,x]$.  We'll expand around $0$, so $x_0=0$ (we do this because we *know* that $e^0=1$; we could choose a different $x_0$ to expand about if we knew what $e^{x_0}$ is).

For this remainder term, the $n$th derivative of $e^{-x}$ is $(-1)^ne^{-x}$, and since our target $x=30$ is positive, we know $\xi \ge 0$ and hence $e^{-\xi} \le 1$, so we can bound this derivative term by $1$ (in absolute value).  We also need to include the coefficient $1/n!$ and the polynomial term $(x-x_0)^n$.

In [6]:
X   = -30

# Taylor's remainder theorem says that the remainder term at the (n-1) step is the nth Taylor series term but evaluated at some \xi
Rem = lambda n : abs( X**n / math.factorial(n) )
for n in range(45):
  print(f'Remainder(n={n:2d}) is {Rem(n):.2e}')
  # When n is large enough, eventually this sequence will start to decrease

Remainder(n= 0) is 1.00e+00
Remainder(n= 1) is 3.00e+01
Remainder(n= 2) is 4.50e+02
Remainder(n= 3) is 4.50e+03
Remainder(n= 4) is 3.38e+04
Remainder(n= 5) is 2.02e+05
Remainder(n= 6) is 1.01e+06
Remainder(n= 7) is 4.34e+06
Remainder(n= 8) is 1.63e+07
Remainder(n= 9) is 5.42e+07
Remainder(n=10) is 1.63e+08
Remainder(n=11) is 4.44e+08
Remainder(n=12) is 1.11e+09
Remainder(n=13) is 2.56e+09
Remainder(n=14) is 5.49e+09
Remainder(n=15) is 1.10e+10
Remainder(n=16) is 2.06e+10
Remainder(n=17) is 3.63e+10
Remainder(n=18) is 6.05e+10
Remainder(n=19) is 9.55e+10
Remainder(n=20) is 1.43e+11
Remainder(n=21) is 2.05e+11
Remainder(n=22) is 2.79e+11
Remainder(n=23) is 3.64e+11
Remainder(n=24) is 4.55e+11
Remainder(n=25) is 5.46e+11
Remainder(n=26) is 6.30e+11
Remainder(n=27) is 7.00e+11
Remainder(n=28) is 7.50e+11
Remainder(n=29) is 7.76e+11
Remainder(n=30) is 7.76e+11
Remainder(n=31) is 7.51e+11
Remainder(n=32) is 7.04e+11
Remainder(n=33) is 6.40e+11
Remainder(n=34) is 5.65e+11
Remainder(n=35) is 4

So if we want 14 digits, let's calculate a bound on the number of Taylor series we need:

In [7]:
tol = 1.0e-14
n = 0
while Rem(n) > tol and n < 1e8 : # this last check is to prevent infinite loops, which are a possibility due to bugs
  n += 1
N = n-1
print( "We need at least", N, "terms, since Rem(n+1) is %.2e" % Rem(N+1) )

We need at least 106 terms, since Rem(n+1) is 9.19e-15


Now, let's sum the Taylor series. We should have an error of 1e-14, right?

In [8]:
e = 0.
for n in range(N+1):
  e += X**n / math.factorial(n)
  if ( not (n % 10 ) ) or ( n > N-10 ):  # save some output...
    print("Iteration %3d"% n, " estimate is %29.15f"% e, "and error is %9.3e" % abs(e - math.exp(X)) )

Iteration   0  estimate is             1.000000000000000 and error is 1.000e+00
Iteration  10  estimate is     121254849.571428596973419 and error is 1.213e+08
Iteration  20  estimate is   85291712210.032180786132812 and error is 8.529e+10
Iteration  30  estimate is  384842612535.907775878906250 and error is 3.848e+11
Iteration  40  estimate is   63336540117.266479492187500 and error is 6.334e+10
Iteration  50  estimate is     878229229.277898311614990 and error is 8.782e+08
Iteration  60  estimate is       1685584.300283375196159 and error is 1.686e+06
Iteration  70  estimate is           622.524572911353516 and error is 6.225e+02
Iteration  80  estimate is             0.055867710574729 and error is 5.587e-02
Iteration  90  estimate is            -0.000084070677839 and error is 8.407e-05
Iteration  97  estimate is            -0.000085530630269 and error is 8.553e-05
Iteration  98  estimate is            -0.000085530022814 and error is 8.553e-05
Iteration  99  estimate is            -0

... but our error is in fact much worse. What went wrong?  Our estimate of the error was due to our mathematical approximation of the exponential, but we didn't include numerical error due to the summation.  We are adding and subtracting a lot of numbers of similar sizes, so we loose precision.

Attempted fix #1: use better summation strategies.  Unfortunately, we'll see that this doesn't quite fix it for us.
- Fix 1: use Python's builtin `fsum`, which does a version of compensated summation (related to [Kahan summation](https://en.wikipedia.org/wiki/Kahan_summation_algorithm), which is a trick that effectively let's us work with twice as many digits)
---



In [9]:
# Fix 1a.
# See https://www.math.ubc.ca/~pwalls/math-python/python/sequences/
series = [ X**n / math.factorial(n) for n in range(N+1) ]  # "lazy", or "iterable". This is a list comprehension
e = math.fsum( series )
abs( e - math.exp(X) )

6.843294459828891e-05

In [36]:
e = math.fsum( reversed(series) ) # no better
# e = sum( series ) # same as fsum (as of Python 3.12, sum() uses Neumaier summation)
# e = np.array(series).sum() % worse
abs( e - math.exp(X) )

6.843294459828891e-05

In [ ]:
# Kahan Summation, though doesn't work (Python might be optimizing?)
e = 0.
accum = 0.
for n in range(N+1):
  a = X**n / math.factorial(n) - accum
  b = e + a
  accum = (b-e) -a
  e = b
  if n > N-4:
    print("Iteration %2d"% n, " estimate is %21.15f"% e, "and error is %9.3e" % abs(e - math.exp(X)) )

Iteration 103  estimate is    -0.000085530164652 and error is 8.553e-05
Iteration 104  estimate is    -0.000085530164246 and error is 8.553e-05
Iteration 105  estimate is    -0.000085530164362 and error is 8.553e-05
Iteration 106  estimate is    -0.000085530164330 and error is 8.553e-05


Attempted fix #2: use some math.
- Fix 2a: split into positive and negative terms, add those separately, then combine at the end. Good idea, but again, doesn't quite fix it.
- Fix 2b: rewrite $e^{-x} = \frac{1}{e^x} = \frac{1}{1+x+x^2/2+\ldots}$.  Finally, we have a good solution!

In [28]:
# Fix 2a
seriesEven = [ X**n / math.factorial(n) for n in range(0,N+1,2) ]
seriesOdd = [ X**n / math.factorial(n) for n in range(1,N+1,2) ]
e = math.fsum( seriesEven ) + math.fsum( seriesOdd )
e = math.fsum( reversed(seriesEven) ) + math.fsum( reversed(seriesOdd) ) # doesn't help much
abs( e - math.exp(X) )

0.0009765625000935762

In [25]:
# Another way to do this, in NumPy:
s = np.array(series)
with np.printoptions(precision=20):
  print( (s[::2].sum() + s[1::2].sum()) - math.exp(X))

0.0009765624999064238


In [ ]:
# Fix 2b
series = [ abs(X)**n / math.factorial(n) for n in range(N+1) ]
e = 1./math.fsum( series )
abs( e - math.exp(X) )